In [1]:
!pip install langchain-community
!pip install "unstructured[all-docs]"
!pip install langchain_ollama
!pip install -U langchain-unstructured
!pip install pymupdf langchain
!pip install chromadb
!pip install -U langchain-openai
!pip install -U langchain langchain-openai chromadb sentence-transformers
!pip install langchain-cohere langchain-vectara
!pip install --upgrade langchain
!pip install langchain langchain-community flashrank
!pip install pytesseract pdf2image
!pip install pytesseract
!pip install pymupdf




   ---------------------------------------- 0.0/11.1 MB ? eta -:--:--
   ------------ --------------------------- 3.4/11.1 MB 22.3 MB/s eta 0:00:01
   --------------------------- ------------ 7.6/11.1 MB 15.2 MB/s eta 0:00:01
   ------------------------------------ --- 10.2/11.1 MB 14.5 MB/s eta 0:00:01
   ---------------------------------------- 11.1/11.1 MB 13.6 MB/s eta 0:00:00
  Attempting uninstall: onnxruntime
    Found existing installation: onnxruntime 1.21.0
    Uninstalling onnxruntime-1.21.0:
      Successfully uninstalled onnxruntime-1.21.0
   ---------------------------------------- 0.0/16.6 MB ? eta -:--:--
   ------ --------------------------------- 2.6/16.6 MB 18.9 MB/s eta 0:00:01
   -------------- ------------------------- 6.0/16.6 MB 16.0 MB/s eta 0:00:01
   -------------------- ------------------- 8.7/16.6 MB 14.5 MB/s eta 0:00:01
   -------------------------- ------------- 11.0/16.6 MB 14.0 MB/s eta 0:00:01
   --------------------------------- ------ 13.9/16.6 MB 1

  error: subprocess-exited-with-error
  
  exit code: 1
  
  [5 lines of output]
  running bdist_wheel
  running build
  running build_ext
  building 'hnswlib' extension
  error: Microsoft Visual C++ 14.0 or greater is required. Get it with "Microsoft C++ Build Tools": https://visualstudio.microsoft.com/visual-cpp-build-tools/
  [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for chroma-hnswlib
ERROR: ERROR: Failed to build installable wheels for some pyproject.toml based projects (chroma-hnswlib)


   ---------------------------------------- 0.0/599.0 kB ? eta -:--:--
   ---------------------------------------- 599.0/599.0 kB 7.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/894.9 kB ? eta -:--:--
   --------------------------------------- 894.9/894.9 kB 39.5 MB/s eta 0:00:00
  Using cached chromadb-1.0.3-cp39-abi3-win_amd64.whl.metadata (7.0 kB)
  Using cached build-1.2.2.post1-py3-none-any.whl.metadata (6.5 kB)
  Using cached chroma_hnswlib-0.7.6.tar.gz (32 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached fastapi-0.115.9-py3-none-any.whl.metadata (27 kB)
  Using cached uvicorn-0.34.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached posthog-3.23.0-py2.py3-none-any.

  error: subprocess-exited-with-error
  
  exit code: 1
  
  [5 lines of output]
  running bdist_wheel
  running build
  running build_ext
  building 'hnswlib' extension
  error: Microsoft Visual C++ 14.0 or greater is required. Get it with "Microsoft C++ Build Tools": https://visualstudio.microsoft.com/visual-cpp-build-tools/
  [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for chroma-hnswlib
ERROR: ERROR: Failed to build installable wheels for some pyproject.toml based projects (chroma-hnswlib)


In [48]:
!pip install requests


In [53]:
!pip install -U langchain-ollama


In [13]:
# --- Step 1: Imports ---

import os
from langchain.chat_models import ChatOpenAI
from langchain.docstore.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_unstructured import UnstructuredLoader
from langchain_community.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter


In [15]:
# --- Step 2: EXTRACTING TEXT FROM DIGITAL/SCANNED PDF & CONVERTING TEXT INTO DOCUMENTS OBJECT ---

import fitz  # PyMuPDF
from pdf2image import convert_from_path
import pytesseract
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document


def extract_text_from_pdf(pdf_path):
    text = ""
    try:
        # Try direct text extraction (digital PDFs)
        doc = fitz.open(pdf_path)
        for page in doc:
            page_text = page.get_text().strip()
            if page_text:
                text += page_text + "\n"
        doc.close()

        if text.strip():
            return text
    except Exception as e:
        print(f"Digital extraction failed: {e}")

    # Fallback to OCR for scanned PDFs
    print("Falling back to OCR extraction...")
    images = convert_from_path(pdf_path)
    for img in images:
        ocr_text = pytesseract.image_to_string(img)
        text += ocr_text + "\n"

    return text

# --- Load and extract text ---
pdf_path = "The Alchemist by Paulo Coelho-1.pdf"
all_text = extract_text_from_pdf(pdf_path)

# --- Split text into chunks ---
splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=100)
chunks = splitter.split_text(all_text)
print("Chunks created:", len(chunks))

# --- Convert chunks to LangChain Documents ---
texts = [Document(page_content=chunk) for chunk in chunks if chunk.strip()]
print("Texts created:", len(texts))


Chunks created: 225
Texts created: 225


In [25]:
!pip install chromadb

In [36]:
# --- Step 3:Embedding chunks using HuggingFace and storing it on chroma vector db  ---
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings

embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")

chroma_db_path = "./alchemist_chroma"

# Create and persist
vectorstore = Chroma.from_documents(texts, embedding_model, persist_directory=chroma_db_path)
vectorstore.persist()

print("✅ Vector store created and saved!")


✅ Vector store created and saved!


In [30]:
from langchain_openai import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain.retrievers.multi_query import MultiQueryRetriever

In [120]:
from langchain_ollama import OllamaLLM

ollama_llm = OllamaLLM(model="llama3.2:1b", temperature=0.7)


In [121]:
# --- Step : MultiQueryRetriever ---
multi_query_retriever = MultiQueryRetriever.from_llm(
    retriever=vectorstore.as_retriever(),
    llm=ollama_llm
)

# --- Step : Memory Setup ---
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True,
    input_key="question",
    output_key="result"
)

# --- Step : Final QA Chain ---
qa_chain = ConversationalRetrievalChain.from_llm(
    llm=ollama_llm,
    retriever=multi_query_retriever,
    memory=memory,
    return_source_documents=True,
    output_key="result"
)


In [124]:
# --- Step 5: Ask Something ---
question = "What is Santiago’s recurring dream about, and how does it influence his journey?"
response = qa_chain.invoke({"question": question})
print("Answer:\n", response["result"])

Answer:
 I don't know. The passage doesn't explicitly state what Santiago's recurring dream about is or how it influences his journey.


What is Santiago’s recurring dream about, and how does it influence his journey?
Who is Melchizedek in The Alchemist, and what role does he play in Santiago’s life?
